# Marketing Analytics

by Harshit Sharma and Sai Vineeth Kaza 

## Introduction
The aim of the project is to Analyse the Marketing Dataset and propose data driven solutions in order to improve marketing campaign results. The solution involves implementing 3 sections namely: 

**Section 01: Exploratory Data Analysis**  
* Are there any null values or outliers? How will you wrangle/handle them?
* Are there any variables that warrant transformations?
* Are there any useful variables that you can engineer with the given data?
* Do you notice any patterns or anomalies in the data? Can you plot them?

**Section 02: Statistical Analysis**  
* What factors are significantly related to the number of store purchases?
* People who spent an above average amount on gold in the last 2 years would have more in store purchases. Justify or refute this statement using an appropriate statistical test. 
* Fish has Omega 3 fatty acids which are good for the brain. Accordingly, do "Married PhD candidates" have a significant relation with amount spent on fish? What other factors are significantly related to amount spent on fish?
* Is there a significant relationship between geographical regional and success of a campaign?

**Section 03: Data Visualization**  
* Which products are performing best?
* Which channels are underperforming?

## Dataset

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
%matplotlib inline
sns.set_context('notebook')
sns.set_style('whitegrid')
sns.set_palette('Blues_r')

# turn off warnings for final notebook
import warnings
warnings.filterwarnings('ignore')

# load dataset
df = pd.read_csv('../input/marketing-data/marketing_data.csv')
print(df.info())
df.head()

* Clean up column names
* Transform selected columns to numeric format:
    - `Income` to float

In [2]:
# clean up column names that contain whitespace
df.columns = df.columns.str.replace(' ', '')

# transform Income column to a numerical
df['Income'] = df['Income'].str.replace('$', '')
df['Income'] = df['Income'].str.replace(',', '').astype('float')

In [3]:
df.head()

# Section 01: Exploratory Data Analysis

### Are there any null values or outliers? How will you wrangle/handle them?

In [4]:
# null values
df.isnull().sum().sort_values(ascending=False)

* The feature `Income` contains 24 null values
* To decide the imputation function, we need to know the distribution of Income feature

In [5]:
plt.figure(figsize=(8,4))
sns.distplot(df['Income'], hist=True)
plt.title('Income distribution', size=16)
plt.ylabel('count');

In [6]:
df['Income'].plot(kind='box', figsize=(3,4), patch_artist=True)

* Impute null values in `Income`, using median value

In [7]:
df['Income'] = df['Income'].fillna(df['Income'].median())

#### Outliers

* Identify features containing outliers:
    - Multiple features contain outliers
    - Only that likely indicate data entry errors are `Year_Birth <= 1900`

In [8]:
# select columns to plot
df_to_plot = df.drop(columns=['ID', 'AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'Response', 'Complain']).select_dtypes(include=np.number)

# subplots
df_to_plot.plot(subplots=True, layout=(4,4), kind='box', figsize=(12,14), patch_artist=True)
plt.subplots_adjust(wspace=0.5);

* Remove rows where `Year_Birth <= 1900`:

In [9]:
df = df[df['Year_Birth'] > 1900].reset_index(drop=True)

plt.figure(figsize=(3,4))
df['Year_Birth'].plot(kind='box', patch_artist=True);

## Are there any variables that warrant transformations?
`Dt_Customer` column should be transformed to datetime format

In [10]:
df.info()

* Transform `Dt_Customer` to datetime:

In [11]:
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'])

### Are there any useful variables that you can engineer with the given data?
* Following columns can be engineered
    - The total number of dependents in the home ('Dependents') can be engineered from the sum of 'Kidhome' and 'Teenhome'
    - The year of becoming a customer ('Year_Customer') can be engineered from 'Dt_Customer'
    - The total amount spent ('TotalMnt') can be engineered from the sum of all features containing the keyword 'Mnt'
    - The total purchases ('TotalPurchases') can be engineered from the sum of all features containing the keyword 'Purchases'

In [12]:
list(df.columns)

* Perform feature engingeering as outlined in notes above:

In [13]:
# Dependents
df['Dependents'] = df['Kidhome'] + df['Teenhome']

# Year becoming a Customer
df['Year_Customer'] = pd.DatetimeIndex(df['Dt_Customer']).year

# Total Amount Spent
mnt_cols = [col for col in df.columns if 'Mnt' in col]
df['TotalMnt'] = df[mnt_cols].sum(axis=1)

# Total Purchases
purchases_cols = [col for col in df.columns if 'Purchases' in col]
df['TotalPurchases'] = df[purchases_cols].sum(axis=1)

In [14]:
# calculate correlation matrix
corrs = df.drop(columns='ID').select_dtypes(include=np.number).corr(method = 'kendall')

# plot clustered heatmap of correlations
sns.clustermap(corrs, cbar_pos=(-0.05, 0.8, 0.05, 0.18), cmap='coolwarm', center=0);

- Finding correlated features

In [15]:
# Finding the mutually correlated features
correlated_features = set()
for i in range(len(corrs.columns)):
    for j in range(i):
        if abs(corrs.iloc[i, j]) > 0.6:
            colname = corrs.columns[i]
            correlated_features.add(colname)

In [16]:
correlated_features

In [17]:
correlated_features.remove('NumStorePurchases')
correlated_features.remove('TotalPurchases')

Plot illustrating the effect of high income on spending:

In [18]:
sns.lmplot(x='Income', y='TotalMnt', data=df[df['Income'] < 200000], line_kws={'color': 'red'});

Plot illustrating negative effect of having dependents (kids & teens) on spending:

In [19]:
plt.figure(figsize=(4,4))
sns.boxplot(x='Dependents', y='TotalMnt', data=df, palette='Oranges');

Plot illustrating positive effect of having dependents (kids & teens) on number of deals purchased:

In [20]:
plt.figure(figsize=(4,4))
sns.boxplot(x='Dependents', y='NumDealsPurchases', data=df, palette='Oranges');

* Investigate anomaly: 
    - Number of web visits in the last month is not positively correlated with number of web purchases
    - Instead, it is positively correlated with the number of deals purchased, suggesting that deals are an effective way of stimulating purchases on the website

In [21]:
sns.lmplot(x='NumWebVisitsMonth', y='NumWebPurchases', data=df, line_kws={'color': 'red'});

In [22]:
sns.lmplot(x='NumWebVisitsMonth', y='NumDealsPurchases', data=df, line_kws={'color': 'red'});

# Section 02: Statistical Analysis
### What factors are significantly related to the number of store purchases?  

In [23]:
# Investigating the target variable
plt.figure(figsize=(8,3))
sns.distplot(df['NumStorePurchases'], kde=False, hist=True, bins=12)
plt.title('NumStorePurchases distribution', size=16)
plt.ylabel('count');

In [24]:
# drop unique ID
df_orig = df.copy()
df.drop(columns=['ID', 'Dt_Customer'], inplace=True)

# removing correlated features
df.drop(columns=correlated_features, inplace=True)

In [25]:
df.columns

In [26]:
# one-hot encoding of categorical features
from sklearn.preprocessing import OneHotEncoder

# get categorical features and review number of unique values
cat = df.select_dtypes(exclude=np.number)

# use one hot encoder
enc = OneHotEncoder(sparse=False).fit(cat)
cat_encoded = pd.DataFrame(enc.transform(cat))
cat_encoded.columns = enc.get_feature_names(cat.columns)

# merge with numeric data
num = df.drop(columns=cat.columns)
df2 = pd.concat([cat_encoded, num], axis=1)
df2.columns=["Education_2n_Cycle"] + list(df2.columns[1:]) 
df2.head()

* Fit linear regression model to the data

In [27]:
# 1. Linear regression using statsmodels
import statsmodels.formula.api as smf
all_columns = list(df2.columns)
all_columns.remove("NumStorePurchases")
all_columns = '+'.join(['_'.join(x.split()) for x in all_columns])
model = smf.ols(formula=' NumStorePurchases ~ {}'.format(all_columns), data=df2).fit()

In [28]:
model.summary()

In [29]:
[x[0] for x in sorted(list(model.pvalues.items()), key=lambda x:x[1]) if x[1] < 0.05]

In [30]:
var_p_values = sorted(list(zip(model.pvalues.index, model.pvalues.values)), key=lambda x:x[1])
var = [x[0] for x in var_p_values]
p_values = [x[1] for x in var_p_values]

In [31]:
plt.figure(figsize=(16,8))
ax=sns.barplot(var, p_values)
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)
ax.axhline(0.05, ls='--', c='red')

* Fit Poisson regression model to the data

In [58]:
import statsmodels.api as sm
from sklearn.linear_model import PoissonRegressor
from sklearn import metrics# Perform train - test split
from sklearn.model_selection import train_test_split

X_PR = df2.drop(columns='NumStorePurchases')
Y_PR = df2['NumStorePurchases']
X_train_PR, X_test_PR, Y_train_PR, Y_test_PR = train_test_split(X_PR, Y_PR, test_size = 0.3, random_state = 42)# Poisson regression model
poisson_training_results = sm.GLM(Y_train_PR, X_train_PR , family =sm.families.Poisson()).fit()

print(poisson_training_results.summary())
print("-"*100)
poisson_predictions = poisson_training_results.get_prediction(X_test_PR)
predictions_summary_frame = poisson_predictions.summary_frame()
print(predictions_summary_frame)
print("-"*100)
summary = poisson_training_results.summary()


def results_summary_to_dataframe(results):
    pvals = results.pvalues
    coeff = results.params
    std_err = results.bse 
    results_df = pd.DataFrame({"pvals":pvals,
                                "coeff":coeff,
                                "std_err": std_err})
    results_df = results_df[["coeff","pvals","std_err"]]
    return results_df

results = results_summary_to_dataframe(poisson_training_results)
results_1 = results.reset_index()
results_1.columns = ['variables','coeff','pvals','std_err']
results_pr = results_1[['variables','pvals','std_err']]
results_pr.columns = ['variables','pvals_PR','std_err_PR']
var_p_values = sorted(results_pr.values, key=lambda x:x[1])
var = [x[0] for x in var_p_values]
p_values = [x[1] for x in var_p_values]
plt.figure(figsize=(16,8))

ax=sns.barplot(var, p_values)
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)
ax.axhline(0.05, ls='--', c='red')

### People who spent an above average amount on gold in the last 2 years would have more in store purchases. Justify or refute this statement using an appropriate statistical test

In [32]:
above_average_df = df[df['MntGoldProds']>df['MntGoldProds'].mean()]
below_average_df = df[df['MntGoldProds']<=df['MntGoldProds'].mean()]

In [33]:
sns.boxplot(above_average_df['NumStorePurchases'], orient="v")
sns.boxplot(below_average_df['NumStorePurchases'], orient="v")

In [34]:
from scipy.stats import ttest_ind
pval = ttest_ind(above_average_df['NumStorePurchases'], below_average_df['NumStorePurchases'])
pval

In [35]:
sns.lmplot(x='MntGoldProds', y='NumStorePurchases', data = df);

* Perform Kendall correlation analysis

In [36]:
from scipy.stats import kendalltau, spearmanr, pearsonr

kendall_corr = kendalltau(x=df['MntGoldProds'], y=df['NumStorePurchases'])

# print results
print('Kendall correlation (tau): ', kendall_corr.correlation)
print('Kendall p-value: ', kendall_corr.pvalue)

In [37]:
plt.hist(df['MntGoldProds'])
plt.xlabel('MntGoldProds')
plt.ylabel('Frequency')

In [38]:
spearmanr(df['MntGoldProds'], df['NumStorePurchases'])

### Fish has Omega 3 fatty acids which are good for the brain. Accordingly, do "Married PhD candidates" have a significant relation with amount spent on fish? 

In [39]:
# sum the marital status and phd dummy variables - the Married+PhD group will have value of 2
df2['Married_PhD'] = df2['Marital_Status_Married'] + df2['Education_PhD']
df2['Married_PhD'] = df2['Married_PhD'].replace({2:'Married-PhD', 1:'Other', 0:'Other'})

# plot MntFishProducts between Married-PhD and others
plt.figure(figsize=(4,7))
sns.boxplot(x='Married_PhD', y='MntFishProducts', data=df2, palette='Oranges');

In [40]:
# independent t-test p-value
from scipy.stats import ttest_ind
pval = ttest_ind(df2[df2['Married_PhD'] == 'Married-PhD']['MntFishProducts'], df2[df2['Married_PhD'] == 'Other']['MntFishProducts'])
print("t-test p-value: ", pval)

In [41]:
# now drop the married-phD column created above, to include only the original variables in the analysis below
df2.drop(columns='Married_PhD', inplace=True)

### Is there a significant relationship between geographical regional and success of a campaign?
Perform logistic regression for each the campaigns and plot the results

In [42]:
# df['Country_code'] = df['Country'].replace({'SP': 'ESP', 'CA': 'CAN', 'US': 'USA', 'SA': 'ZAF', 'ME': 'MEX'})
# success of campaigns by country code
df_cam = df[['Country', 'AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'Response']].melt(
    id_vars='Country', var_name='Campaign', value_name='Accepted (%)')
df_cam = pd.DataFrame(df_cam.groupby(['Country', 'Campaign'])['Accepted (%)'].mean()*100).reset_index(drop=False)

# rename the campaign variables so they're easier to interpret
df_cam['Campaign'] = df_cam['Campaign'].replace({'AcceptedCmp1': '1',
                                                'AcceptedCmp2': '2',
                                                'AcceptedCmp3': '3',
                                                'AcceptedCmp4': '4',
                                                'AcceptedCmp5': '5',
                                                 'Response': 'Most recent'
                                                })

* Statistical summary of regional effects on campaign success:
    - Methodology: Performed logistic regression for Campaign Accepted by Country, reporting Chisq p-value for overall model.
    - Findings: The regional differences in advertising campaign success are statistically significant.

In [43]:
df_cam.head(10)

In [44]:
# calculate logistic regression p-values for campaign acceptance ~ country using generalized linear model
import statsmodels.formula.api as smf
import statsmodels as sm
from scipy import stats

## get the data of interest for glm
df_cam_wide = df[['Country', 'AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'Response']]

## to store statistics results
stat_results = []

## perform glm
for col in df_cam_wide.drop(columns='Country').columns:
    this_data = df_cam_wide[['Country', col]]
    
    # define formula
    formula = col+'~Country'
    
    # logistic regression (family=binomial)
    model = smf.glm(formula = formula, data=this_data, family=sm.genmod.families.Binomial())
    result = model.fit()
    
    # get chisquare value for overall model (CampaignAccepted ~ Country) and calculate p-value
    chisq = result.pearson_chi2
    pval = stats.distributions.chi2.sf(chisq , 7) # Df Model = 7 degrees of freedom when you run result.summary()
     
    # append to stat_results
    stat_results.append(pval)
    
    # print stat summary for entire model
    print(result.summary())
    
## check results
print("\nChisq p-values: ", stat_results)

In [45]:
## bar graphs
g = sns.FacetGrid(df_cam, col='Campaign', col_wrap=3)
g.map(sns.barplot, 'Country', 'Accepted (%)')
for ax, pval in zip(g.axes.flat, stat_results):
    ax.text(0, 65, "Chisq p-value: "+str(pval), fontsize=9)

# Section 03: Data Visualization

### Which products are performing best?

In [47]:
spending = pd.DataFrame(round(df_orig[mnt_cols].mean(), 1), columns=['Average']).sort_values(by='Average').reset_index()

# plot
ax = sns.barplot(x='Average', y='index', data=spending, palette='Blues')
plt.ylabel('Amount spent on...')

## add text labels for each bar's value
for p,q in zip(ax.patches, spending['Average']):
    ax.text(x=q+40,
            y=p.get_y()+0.5,
            s=q,
            ha="center") ;

### Which channels are underperforming?

In [49]:
df_orig.columns

In [50]:
channel_cols = ['NumDealsPurchases', 'NumWebPurchases','NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth']
channels = pd.DataFrame(round(df_orig[channel_cols].mean(), 1), columns=['Average']).sort_values(by='Average').reset_index()

# plot
ax = sns.barplot(x='Average', y='index', data=channels, palette='Blues')
plt.ylabel('Number of...')

## add text labels for each bar's value
for p,q in zip(ax.patches, channels['Average']):
    ax.text(x=q+0.8,
            y=p.get_y()+0.5,
            s=q,
            ha="center") ;

# Conclusion

Summary of suggested actions:
* The most successful advertising campaign was the most recent campaign (column name: Response), and was particularly successful in Mexico (>60% acceptance rate!)
Suggested action: Conduct future advertising campaigns using the same model recently implemented in Mexico.

* The most successful products are wines and meats (i.e. the average customer spent the most on these items).
Suggested action: Focus advertising campaigns on boosting sales of the less popular items.


* The underperforming channels are deals and catalog purchases (i.e. the average customer made the fewest purchases via these channels).
The best performing channels are web and store purchases (i.e. the average customer made the most purchases via these channels).
Suggested action: Focus advertising campaigns on the more successful channels, to reach more customers.

* Linear and Poisson regression models considered 'TotalPurchases', 'NumCatalogPurchases', 'NumWebPurchases', 'NumDealsPurchases', 'NumWebVisitsMonth' and 'Kidhome' features as statistically significant. 
Suggested action: Focus advertising campaigns to increase the sales in the above mentioned channels and to attract the kids.